In [1]:
# Date - May 05 2020
# Explore Finance Data Simulated Credit Card Fraud
# Data Source - https://www.kaggle.com/ntnu-testimon/paysim1

# Ensure that all of the specified libraries are installed.

dbutils.library.installPyPI("mlflow")
dbutils.library.restartPython()

In [2]:
# Configure MLflow Experiment
mlflow_experiment_id = 866112

# Including MLflow
import mlflow
import mlflow.spark

import os
print("MLflow Version: %s" % mlflow.__version__)


MLflow Version: 1.8.0

In [3]:
# Create df DataFrame which contains our simulated financial fraud detection dataset
df = spark.sql("select step, type, amount, nameOrig, oldbalanceOrg, newbalanceOrig, nameDest, oldbalanceDest, newbalanceDest from sim_fin_fraud_detection")

df

df.printSchema()


root
-- step: string (nullable = true)
-- type: string (nullable = true)
-- amount: string (nullable = true)
-- nameOrig: string (nullable = true)
-- oldbalanceOrg: string (nullable = true)
-- newbalanceOrig: string (nullable = true)
-- nameDest: string (nullable = true)
-- oldbalanceDest: string (nullable = true)
-- newbalanceDest: string (nullable = true)

In [4]:
# Calculate the differences between originating and destination balances
df = df.withColumn("orgDiff", df.newbalanceOrig - df.oldbalanceOrg).withColumn("destDiff", df.newbalanceDest - df.oldbalanceDest)

# Create temporary view
df.createOrReplaceTempView("financials")

In [5]:
%sql

select type, count(1) from sim_fin_fraud_detection group by type


type,count(1)
TRANSFER,532909
CASH_IN,1399284
CASH_OUT,2237500
PAYMENT,2151495
DEBIT,41432


In [6]:
%sql

select type, sum(amount) from sim_fin_fraud_detection group by type


type,sum(CAST(amount AS DOUBLE))
TRANSFER,4.8529198726316675E11
CASH_IN,2.3636739191245865E11
CASH_OUT,3.944129952244882E11
PAYMENT,2.809337113837033E10
DEBIT,2.2719922127999964E8


In [7]:
from pyspark.sql import functions as F

# Rules to Identify Known Fraud-based
df = df.withColumn("label", 
                   F.when(
                     (
                       (df.oldbalanceOrg <= 56900) & (df.type == "TRANSFER") & (df.newbalanceDest <= 105)) | ( (df.oldbalanceOrg > 56900) & (df.newbalanceOrig <= 12)) | ( (df.oldbalanceOrg > 56900) & (df.newbalanceOrig > 12) & (df.amount > 1160000)
                           ), 1
                   ).otherwise(0))

# Calculate proportions
fraud_cases = df.filter(df.label == 1).count()
total_cases = df.count()
fraud_pct = 1.*fraud_cases/total_cases

# Provide quick statistics
print("Based on these rules, we have flagged %s (%s) fraud cases out of a total of %s cases." % (fraud_cases, fraud_pct, total_cases))

# Create temporary view to review data
df.createOrReplaceTempView("financials_labeled")

Based on these rules, we have flagged 255287 (0.04012293677761677) fraud cases out of a total of 6362620 cases.

In [8]:
%sql
select label, count(1) as `Transactions`, sum(amount) as `Total Amount` from financials_labeled group by label

label,Transactions,Total Amount
1,255287,1.263467543146704E11
0,6107333,1.0180461904451278E12


In [9]:
%sql
-- where sum(destDiff) >= 10000000.00
select nameOrig, nameDest, label, TotalOrgDiff, TotalDestDiff
  from (
     select nameOrig, nameDest, label, sum(OrgDiff) as TotalOrgDiff, sum(destDiff) as TotalDestDiff 
       from financials_labeled 
      group by nameOrig, nameDest, label 
     ) a
 where TotalDestDiff >= 1000000
 limit 100

nameOrig,nameDest,label,TotalOrgDiff,TotalDestDiff
C752686443,C1816757085,0,-51178.0,1037297.459999999
C1034899044,C306206744,0,0.0,2308448.1800000006
C1337803451,C1778801068,0,0.0,3155647.91
C1570013189,C1945802665,1,-63916.0,1179067.2799999998
C1379731404,C413177525,0,0.0,2375262.37
C608080514,C2006081398,1,-101396.29,3208776.46
C1567963311,C1141049797,0,136177.75999999978,1243657.54
C582291919,C461640598,0,0.0,3381134.63
C689413511,C1988852187,0,0.0,1161908.05
C122861681,C1754722089,0,-22291.0,1512938.6399999997


In [10]:
%sql
select type, label, count(1) as `Transactions` from financials_labeled group by type, label

type,label,Transactions
PAYMENT,0,2150909
CASH_OUT,0,2052932
DEBIT,1,31
TRANSFER,1,70100
CASH_OUT,1,184568
PAYMENT,1,586
DEBIT,0,41401
CASH_IN,0,1399282
TRANSFER,0,462809
CASH_IN,1,2


In [11]:
# Initially split our dataset between training and test datasets
(train, test) = df.randomSplit([0.8, 0.2], seed=12345)

# Cache the training and test datasets
train.cache()
test.cache()

# Print out dataset counts
print("Total rows: %s, Training rows: %s, Test rows: %s" % (df.count(), train.count(), test.count()))

Total rows: 6362620, Training rows: 5090311, Test rows: 1272309

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler

#from pyspark.ml.feature import HashingTF, Tokenizer

from pyspark.ml.classification import DecisionTreeClassifier

# Encodes a string column of labels to a column of label indices   
#Tokenizer
indexer = StringIndexer(inputCol = "type", outputCol = "typeIndexed")

# VectorAssembler is a transformer that combines a given list of columns into a single vector column 
#HashingTF
va = VectorAssembler(inputCols = ["typeIndexed", "amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest", "orgDiff", "destDiff"], outputCol = "features")

#hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

# Using the DecisionTree classifier model
dt = DecisionTreeClassifier(labelCol = "label", featuresCol = "features", seed = 54321, maxDepth = 5)

# Create our pipeline stages
#pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
pipeline = Pipeline(stages=[indexer, va, dt])


In [13]:
# View the Decision Tree model (prior to CrossValidator)
model = pipeline.fit(train)
#display(dt_model.stages[-1])

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o975.transform.
: java.lang.IllegalArgumentException: Data type string of column amount is not supported.
Data type string of column oldbalanceOrg is not supported.
Data type string of column newbalanceOrig is not supported.
Data type string of column oldbalanceDest is not supported.
Data type string of column newbalanceDest is not supported.
	at org.apache.spark.ml.feature.VectorAssembler.transformSchema(VectorAssembler.scala:171)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:74)
	at org.apache.spark.ml.feature.VectorAssembler.transform(VectorAssembler.scala:88)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 IllegalArgumentException Traceback (most recent call last)
 <command-3477732143359000> in <module> 
 1 # View the Decision Tree model (prior to CrossValidator) 
 ----> 2 model = pipeline . fit ( train ) 
 3 #display(dt_model.stages[-1]) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 105 if isinstance ( stage , Transformer ) : 
 106 transformers . append ( stage ) 
 --> 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 109 model = stage . fit ( dataset ) 

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 171 return self . copy ( params ) . _transform ( dataset ) 
 172 else : 
 --> 173 return self . _transform ( dataset ) 
 174 else : 
 175 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _transform (self, dataset) 
 310 def _transform ( self , dataset ) : 
 311 self . _transfer_params_to_java ( ) 
 --> 312 return DataFrame ( self . _java_obj . transform ( dataset . _jdf ) , dataset . sql_ctx ) 
 313 
 314 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 77 raise QueryExecutionException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 78 if s . startswith ( 'java.lang.IllegalArgumentException: ' ) : 
 ---> 79 raise IllegalArgumentException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 80 raise 
 81 return deco

 IllegalArgumentEx